In [18]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import re
import sys
import time

import numpy as np
import pandas as pd
import os
import psycopg2

from tqdm import tqdm

In [19]:
try:
    skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
except:
    skiplist = []

In [20]:
skiplist

[115922,
 119509,
 119510,
 18780,
 25273,
 28631,
 29112,
 31395,
 32583,
 33032,
 33276,
 115922,
 119509,
 119510,
 25273,
 115922,
 119509,
 119510,
 25273,
 67583,
 68459,
 80854,
 100,
 1000,
 100000,
 100024,
 10008,
 100082,
 100086,
 100088,
 100092,
 100094,
 100098,
 100100,
 100122,
 100143,
 100151,
 100153,
 100160,
 116398,
 10018,
 100186,
 100192,
 100195,
 100209,
 100221,
 100222,
 100230,
 100236,
 104692,
 100245,
 100247,
 10026,
 100263,
 100271,
 100274,
 100279,
 100281,
 100297,
 10031,
 100333,
 100334,
 100337,
 10035,
 100358,
 10036,
 100367,
 100369,
 10037,
 100371,
 100387,
 100388,
 100390,
 100418,
 101909,
 102057,
 102058,
 102059,
 10206,
 102060,
 102061,
 102062,
 102063,
 102064,
 102065,
 102066,
 102067,
 102068,
 102069,
 102070,
 102071,
 102072,
 102073,
 102074,
 102075,
 102076,
 102077,
 102078,
 102079,
 10208,
 102080,
 102081,
 102082,
 102083,
 102084,
 102085,
 102086,
 102087,
 102088,
 102089,
 10209,
 102090,
 102091,
 102092,
 1

In [21]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

cur = conn.cursor()

In [22]:
command = """
SELECT doc_id,doc_url
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
LIMIT 5
""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

In [23]:
existing

[(109162,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1454964.shtml'),
 (109163,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1453237.shtml'),
 (109164,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1452946.shtml'),
 (109165,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1452342.shtml'),
 (109166,
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1451919.shtml')]

In [24]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT count(doc_id)
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}

""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

In [25]:
existing

[(102912,)]

In [26]:
command = """
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'documents'
"""

cur.execute(command)

columns = cur.fetchall()

In [27]:
columns

[('doc_id',),
 ('doc_country',),
 ('doc_language',),
 ('doc_page_name',),
 ('doc_url',),
 ('doc_nav',),
 ('doc_title',),
 ('doc_date',),
 ('doc_html',),
 ('doc_content',),
 ('doc_skip',)]

In [28]:
def html(url):
    try:
        driver.get(url)
        html = driver.page_source
        if type(html) == str:
            return(html)
        else:
            return('ERROR')
    except:
        return('ERROR')

In [29]:
def headlessDriver():
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options, executable_path="geckodriver.exe")
    return(driver)

In [30]:
def buildEntry(index_value,url):
    t_start = time.time()
    
    index = index_value
    pageSource = html(url)
#     pageDate = str()
    pageText = str()
    if pageSource == 'ERROR':
#         pageDate = 'ERROR'
        pageText = 'ERROR'
    else:
        try:
            soup=BeautifulSoup(pageSource, 'lxml')
#             pageDate = soup.find("span", {"id": "News_Body_Time"}).text
#             if pageDate == '':
#                 pageDate = None
            pageText = soup.find(class_='content').text
            if pageText == '':
                pageText = None
        except:
#             pageDate = 'ERROR'
            pageText = 'ERROR'
    
    entry = (pageSource,
#              pageDate,
             pageText,
             index)
    
    t_end = time.time()
    
    duration = t_end - t_start
    
    if duration >= 10:
        time.sleep(np.random.randint(5,15))
    
#   print(duration)          
    return(entry)

In [31]:
conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                    database="postgres",
                    user="tokyo3",
                    password="evangelion")

cur = conn.cursor()

command = """
SELECT *
FROM documents
WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
LIMIT 10
""".format(tuple(skiplist))

cur.execute(command)

existing = cur.fetchall()

cur.close()
conn.close()

In [ ]:
existing

[(109162,
  '汤加',
  'zh',
  '驻外报道',
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1454964.shtml',
  '首页_国家和组织_国家（地区）_大洋洲_汤加_驻外报道_',
  '驻汤加大使黄华光出席汤加政府和汤中友协送行晚宴(2017-04-20)',
  datetime.date(2017, 4, 20),
  None,
  None,
  None),
 (109163,
  '汤加',
  'zh',
  '驻外报道',
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1453237.shtml',
  '首页_国家和组织_国家（地区）_大洋洲_汤加_驻外报道_',
  '驻汤加使馆举办黄华光大使离任招待会(2017-04-13)',
  datetime.date(2017, 4, 13),
  None,
  None,
  None),
 (109164,
  '汤加',
  'zh',
  '驻外报道',
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1452946.shtml',
  '首页_国家和组织_国家（地区）_大洋洲_汤加_驻外报道_',
  '驻汤加大使黄华光向汤加国王陛下辞行(2017-04-12)',
  datetime.date(2017, 4, 12),
  None,
  None,
  None),
 (109165,
  '汤加',
  'zh',
  '驻外报道',
  'https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681790/1206x2_681810/t1452342.shtml',
  '首页_国家和组织_国家（地区）_大洋洲_汤加_驻外报道_',
  '驻汤加大使

In [ ]:
def fullHarvest():
    t = time.time()
    try:
        skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
    except:
        skiplist = []
        
    
    limit = 1
    
    conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

    cur = conn.cursor()
    
    command = """
    SELECT doc_id,doc_url
    FROM documents
    WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
    LIMIT {}
    """.format(tuple(skiplist),limit)

    cur.execute(command)

    existing = cur.fetchall()
#     print(existing)
    cur.close()
    conn.close()
    
#     driver = None
#     driver = headlessDriver()
    submission = [buildEntry(int(x[0]),str(x[1])) for x in existing]
    
    newSubmission = []
    for i in submission:
        if i[1] == 'ERROR':
            skiplist.append(i[2])
        else:
            newSubmission.append(i)
    
    
    pd.Series(skiplist).to_csv('skiplist.csv')
    
    
    sub_len = len(newSubmission)
    
    
    
    
    conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
                        database="postgres",
                        user="tokyo3",
                        password="evangelion")

    cur = conn.cursor()
    sql = """
        UPDATE documents
        SET doc_html = %s,doc_content = %s
        WHERE doc_id = %s;
        """

    cur.executemany(sql,newSubmission)
    conn.commit()
    
    
    command = """
    SELECT COUNT(doc_id)
    FROM documents
    WHERE doc_html IS NULL AND doc_date IS NOT NULL and doc_id NOT IN {}
    """.format(tuple(skiplist))


    cur.execute(command)

    count = cur.fetchall()
    

    cur.close()
    conn.close()
    
    nt = time.time()
    dt = nt - t
    
    
    
    return(len(skiplist),count,dt,sub_len,existing)

    

In [ ]:
counter = 0
driver = headlessDriver()

for i in tqdm(range(10000)):    
    s,r,t,l,e = fullHarvest()
    print("{} : {:.4f}s : {} skips : {} remaining : {} counter".format(i,t,s,r,counter))
    if l == 0:
        print(e)
        counter += 1
        if counter > 10:
            break
        else:
            time.sleep(np.random.randint(15,30))
            driver = headlessDriver()
            
        


  0%|          | 0/10000 [00:00<?, ?it/s]

0 : 43.8468s : 888 skips : [(102911,)] remaining : 0 counter



  0%|          | 1/10000 [00:43<121:47:13, 43.85s/it]

1 : 11.3573s : 888 skips : [(102910,)] remaining : 0 counter



  0%|          | 2/10000 [00:55<94:42:29, 34.10s/it] 

...


  2%|▏         | 193/10000 [16:59<11:55:29,  4.38s/it]

193 : 4.5803s : 888 skips : [(102718,)] remaining : 0 counter



  2%|▏         | 194/10000 [17:04<12:05:28,  4.44s/it]

194 : 4.7017s : 888 skips : [(102717,)] remaining : 0 counter



  2%|▏         | 195/10000 [17:08<12:18:22,  4.52s/it]

In [ ]:
# buildEntry(104328,"https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/dyz_681240/1206_681242/1206x2_681262/t1145802.shtml")

In [ ]:
# driver = headlessDriver()
# buildEntry(102358,"""https://www.fmprc.gov.cn/web/gjhdq_676201/gj_676203/nmz_680924/1206_681192/1206x2_681212/t878484.shtml""")

In [ ]:
try:
    skiplist = list(pd.read_csv('skiplist.csv',index_col=0,header=None).iloc[:,0])
except:
    skiplist = []

In [ ]:
len(skiplist)

In [ ]:
skiplist

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()
# command = """
# ALTER TABLE documents 
# ADD COLUMN doc_skip SMALLINT
# """
# cur.execute(command)
# conn.commit()

# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT column_name
# FROM information_schema.columns
# WHERE table_name = 'documents'
# """

# cur.execute(command)

# columns = cur.fetchall()


# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# columns

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# sql = """
#         UPDATE documents
#         SET doc_skip = 1
#         WHERE doc_id IN {};
#         """.format(tuple(skiplist))

# cur.execute(sql)
# conn.commit()



# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# sql = """
#         SELECT * 
#         FROM documents
#         WHERE doc_skip = 1
#         LIMIT 10
#         """

# cur.execute(sql)

# sample = cur.fetchall()

# # extract = cur.fetchall()
# # print(extract)
# cur.close()
# conn.close()

In [ ]:
# sample

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT doc_id,doc_url
# FROM documents
# WHERE doc_url NOT SIMILAR TO '%shtml' AND doc_id NOT IN {}
# """.format(tuple(skiplist))


# cur.execute(command)

# extract = cur.fetchall()
# print(extract)
# cur.close()
# conn.close()

In [ ]:
# submission[-1]

In [ ]:
# for i in submission:
#     print(i[2],'\n')

In [ ]:
# print(len(submission))

In [ ]:
# for i in submission:
#     if i[2] == 'ERROR':
#         print(i[3])

In [ ]:
# newSubmission = []
# for i in submission:
#     if i[1] == 'ERROR':
#         skiplist.append(i[3])
#     else:
#         newSubmission.append(i)

In [ ]:
# pd.Series(skiplist).to_csv('skiplist.csv')

In [ ]:
# pd.Series(skiplist)

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

In [ ]:
# sql = """
#         UPDATE documents
#         SET doc_html = %s,doc_date = %s,doc_content = %s
#         WHERE doc_id = %s;
#         """

# cur.executemany(sql,newSubmission)
# conn.commit()

# cur.close()
# conn.close()

In [ ]:
# existing

In [ ]:
# len(existing)

In [ ]:
# listCheck = [x[0] for x in existing]

In [ ]:
# listCheck

In [ ]:
# tuple(listCheck)

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT *
# FROM documents
# WHERE doc_id IN {}
# """.format(tuple(listCheck))

# cur.execute(command)

# extract = cur.fetchall()

# cur.close()
# conn.close()

In [ ]:
# conn = psycopg2.connect(host="chinaquant.cofdbub8go3o.eu-west-1.rds.amazonaws.com",
#                         database="postgres",
#                         user="tokyo3",
#                         password="evangelion")

# cur = conn.cursor()

# command = """
# SELECT COUNT(doc_id)
# FROM documents
# WHERE doc_html IS NULL AND doc_date IS NULL and doc_id NOT IN {}
# """.format(tuple(skiplist))


# cur.execute(command)

# extract = cur.fetchall()

# cur.close()
# conn.close()

In [ ]:
# extract